# Project proposal

---

Group name: **EDART Collective**

Projekt: **End-to-End Darts Analytics – Prognose des FDI-Ratings**

Semester: **Wintersemester 2025/26**

---

*Lead*: Simon H. – Data Engineering, Modellierung & Deployment

## Introduction

Professioneller Darts-Sport generiert inzwischen Millionenpublikum und umfangreiche Statistikfeeds. Dennoch greifen aktuelle Rankings wie Order of Merit oder Elo-artige Systeme nur begrenzt die kurzfristige Form sowie psychologische Faktoren auf. Unser Projekt **„End-to-End Darts Analytics“** adressiert diese Lücke, indem wir das proprietäre *Future Dart Intelligence (FDI) Rating* von Darts Orakel datengetrieben prognostizieren. Frühere Analysen (z. B. Lucas 2023, *Journal of Sports Analytics*) zeigen, dass kombinierte Skill- und Momentum-Metriken die Vorhersagequalität gegenüber klassischen Durchschnittswerten um >10 % verbessern können – diese Evidenz motiviert unsere Forschungsfrage.

**Forschungsfrage:**
> Wie genau lässt sich das FDI-Rating professioneller Spieler:innen prognostizieren, wenn Hard Facts (Scores, Checkout, First-9) mit Soft Facts (log Preisgeld, Clutch-Win %, Tour-Card-Historie) gemeinsam modelliert werden?

**Hypothesen:**
1. Modelle, die Soft-Facts integrieren, senken den RMSE gegenüber rein statistischen Baselines signifikant.
2. Regularisierte Verfahren (Lasso/XGBoost) outperformen lineare Modelle, da Nichtlinearitäten und Interaktionen zwischen Hard-/Soft-Facts relevant sind.
3. Spieler:innen mit stark steigender `log_prize_money`-Trendlinie weisen eine frühzeitige positive Abweichung zum beobachteten FDI auf (Leading Indicator).

## Data description

**Beobachtungseinheiten:**
- *Player-Master*: 2 978 Spieler:innen (Stand Dez 2025), jeweils eine Zeile mit aggregierten Profil- und Performance-Kennzahlen der letzten 12 Monate.
- *Match-/Leg-Facts*: Granulare Tabellen (geplante Erweiterung), jede Zeile = Spieler × Leg inkl. Kontext (Turnier, Runde, Gegner), aktuell für Major Events 2024 in ETL-Aufbau.

**Primärquelle:** [Darts Orakel](https://dartsorakel.com) stellt das FDI-Rating, Score-Durchschnitte, Checkout- und Momentum-Metriken als HTML/JS-Oberfläche bereit. Über eine selbstentwickelte Scraper-Pipeline (Requests + BeautifulSoup + Playwright-Fallback) werden täglich Snapshots gezogen. Das Backend verwendet eine self-hosted PostgreSQL-Instanz (Docker auf Hetzner) zur Persistenz und Versionierung.

**Sekundärquellen & Meta-Daten:**
- PDC/WDF/Wikipedia: Turnierkalender, Preisgeld, World Ranking, Tour-Card-Status.
- PDPA/Press Releases: Psychologische Proxies (Reisebelastung über Turnierorte, Rest Days).

**Erhebung/Originalkurator:** Darts Orakel aggregiert Daten aus offiziellen PDC/WDF-Match Feeds (Darts Data Hub) sowie öffentlich zugänglichen Resultsheets, rechnet daraus das FDI-Rating und stellt APIs für Aggregationen bereit. Unsere Nutzung respektiert die Nutzungsbedingungen (Credit + Rate-Limit), indem wir eigene ETL-Jobs mit throttled Requests betreiben. Sekundärdaten stammen aus öffentlich verfügbaren, versionierten Turnierlisten (z. B. Wikipedia dumps, PDC PDF bulletins).

## Analysis approach

- **Response Variable:** `fdi_rating` (kontinuierlich, Originalskala 0–2100). Wir evaluieren sowohl den absoluten Fehler (RMSE/MAE) als auch Klassifikationen in Quantile (Top 16 / Top 32) als Neben-Auswertung.
- **Explorative Statistik:**
  - Verteilung von `fdi_rating` (Histogramm, KDE, QQ-Plot) zur Prüfung auf Normalität/Schiefe.
  - Scatterplots & Korrelationsmatrix für Schlüssel-Features (`three_dart_avg`, `first9_avg`, `checkout_pct`, `log_prize_money`, `win_pct`).
  - Zeitliche Trends per Rolling Windows (z. B. `rolling_fdi_delta_4w`).
- **Predictor Set:**
  - **Hard Facts:** `three_dart_avg`, `first9_avg`, `checkout_pct`, `first3_avg`, `legs_won_pct`, `highest_checkout`, `order_of_merit_rank`.
  - **Soft/Context:** `log_prize_money`, `win_pct`, `tour_card_years`, `api_rank_delta`, `momentum_score`, `region_cluster`, `age`, `birth_country`, `stage_intensity_index`.
  - **Engineered:** `clutch_win_pct`, `form_streak`, `travel_fatigue_proxy`, `rest_days`, Interaktionen (z. B. `first9_avg * clutch_win_pct`).
- **Modellfamilien:**
  1. Baseline OLS & Ridge (für Interpretierbarkeit, Multikollinearität durch Shrinkage).
  2. Lasso für Feature Selektion + Sparse Solutions.
  3. Gradient Boosted Trees / XGBoost & Random Forest für nichtlineare Effekte und Interaktionen.
  4. Optional: LightGBM + SHAP zur Explainability.
- **Validierung:** 5-fold Cross Validation + zeitlich bewusster Split (Train = Vorjahre, Test = jüngste Quartale) um Look-Ahead-Bias zu vermeiden. Hyperparameter-Tuning via Optuna (uv-managed).

## Data dictionary

| Name | Description | Role | Type | Format |
|---|---|---|---|---|
| `player_id` | Eindeutige ID von Darts Orakel / PDC | ID | nominal | int |
| `player_name` | Klartextname | ID | nominal | string |
| `profile_fdi_rating` | Future Dart Intelligence Score (Target) | response | numeric | float |
| `last_12_months_averages` | Durchschnittlicher 3-Dart-Average (Rolling 12M) | predictor | numeric | float |
| `last_12_months_first_9_averages` | First-9-Average als Momentum-Proxi | predictor | numeric | float |
| `last_12_months_checkout_pcnt` | Checkout-Quote in % | predictor | numeric | float |
| `last_12_months_pcnt_legs_won` | Anteil gewonnener Legs | predictor | numeric | float |
| `profile_total_earnings` | Kumuliertes Preisgeld (Profilkarte) | predictor | numeric | float |
| `profile_season_win_pct` | Saisonale Match-Win-Rate | predictor | numeric | float |
| `profile_tour_card_years` | Anzahl Tour-Card-Jahre | predictor | numeric | int |
| `api_rank` | Rang laut Orakel-API | predictor | ordinal | int |
| `api_overall_stat` | API-Kennzahl (Averages) | predictor | numeric | float |
| `country_code` | Nationskürzel laut Profil | predictor | nominal | string |
| `age` | Alter (soweit im Profil genannt) | predictor | numeric | float |

*Hinweis:* Zusätzliche Match-Level Variablen (z. B. `stage`, `opponent_rank_gap`, `rest_days`) werden im PostgreSQL-Schema geführt und bei Bedarf über Feature Stores in das Modell überführt.

## Explorative Datenanalyse (EDA)

- **Datenbasis:** `player_stats_all.csv` (2 978 Spieler:innen × 32 Spalten). Enthält konsolidierte Profilzweige (`profile_*`) sowie Rolling-12M-KPIs (`last_12_months_*`) und API-Metadaten.
- **Zielgröße:** `profile_fdi_rating` liegt für 2 477 Spieler:innen vor (Fehlwerte bei Akteur:innen ohne offizielles Profil). Mittelwert 1 353 Punkte, SD 183, Range 752–1 972.
- **Kernmetriken:**
  - `last_12_months_averages` median 69.97, 95 %-Quantil 87.4 – deutliche Rechtsschiefe durch Weltklasse-Spieler.
  - `last_12_months_first_9_averages` hoch mit FDI korreliert (ρ≈0.95), ebenso `api_overall_stat` – deutet auf starke Kopplung zwischen Scoring und Rating hin.
  - `profile_total_earnings` median 0 € (Große Basis an Semi-Pros); 95 %-Quantil knapp 90 k €.
- **Top-Performer:** Luke Littler, Gerwyn Price, Luke Humphries, Wessel Nijman, Gian van Veen – jeweils FDI > 1 860, Three-Dart-Averages ≥ 95.
- **Missingness:** Größte Lücken bei `age`, `birth_date`, `profile_tour_card_years` (>2.7 k Fälle) → Im Modell via Imputation / Indicator-Features berücksichtigen.
- **Länderabdeckung:** 70+ Nationen, Spitze England (573), Niederlande (221), Deutschland (207), Australien (102). Länder-Dummies ermöglichen geographische Cluster.
- **Insights:**
  1. Checkout-Quote (ρ≈0.88) liefert starken Zusatznutzen gegenüber reinem Scoring.
  2. Gewinnquoten & Momentum-KPIs unterscheiden Top-16-Cluster (Leg-Win% >60) klar von Longtail (<45).
  3. Tour-Card-Jahre korrelieren leicht negativ (ρ≈−0.15) – jüngere Talente (z. B. Littler) pushen FDI trotz kurzer Historie.

Die folgenden Code-Zellen reproduzieren sämtliche Kennzahlen, Visualisierungen und Tabellen, ausführbar via `uv run jupyter` bzw. VS Code Notebook-UI.